In [1]:
import pandas as pd
import numpy as np

In [2]:
grid = pd.read_csv('./Data/final_game_user_grid_06_07_19.csv')

In [3]:
grid.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1387492 entries, 0 to 1387491
Data columns (total 7 columns):
user_name       1387492 non-null object
game_name       1385813 non-null object
game_genres     1383564 non-null object
language        1387490 non-null object
started_at      1387492 non-null object
viewer_count    1387492 non-null int64
max             1387492 non-null int64
dtypes: int64(2), object(5)
memory usage: 74.1+ MB


In [4]:
grid = grid.dropna()

In [5]:
grid = grid.dropna(how='any',axis=0) 

In [6]:
grid.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1383562 entries, 0 to 1387491
Data columns (total 7 columns):
user_name       1383562 non-null object
game_name       1383562 non-null object
game_genres     1383562 non-null object
language        1383562 non-null object
started_at      1383562 non-null object
viewer_count    1383562 non-null int64
max             1383562 non-null int64
dtypes: int64(2), object(5)
memory usage: 84.4+ MB


In [7]:
#change all NONE, Null, and RETROGAMEPLACEHOLDER genres to 'Other'
grid['game_genres'][grid['game_genres'].str.contains('NONE')==True] = 'Other'
grid['game_genres'][grid['game_genres'].isnull()] = 'Other'
grid['game_genres'][grid['game_genres'].str.contains('RETROGAMEPLACEHOLDER')==True] = 'Other'

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [9]:
# To begin, we're calculating the metric we'll be using to compare users and games. Since we're focused on the streamers, we don't have the traditional "I like this movie so I will rate it a 4.5 out of 5.0" ratings. Instead we're calculating how many viewers each streamer attracted with a particular game compared to their max viewer potential over the last week. For each user, one game is their ultimate streaming "5 out of 5" benchmark, and all other games they play are compared to that one, and normalized to ratings between 1 and 5. We are also using all the genres for each game to pinpoint how successful various genres and games have been for each streamer.

In [10]:
max_value_username = pd.DataFrame(grid.groupby('user_name')['max'].max().reset_index())

In [11]:
max_val_dict = max_value_username.groupby('user_name')['max'].apply(list).to_dict()

In [12]:
grid['max_game'] = grid['user_name'].map(max_val_dict)

In [13]:
grid['max_game_int'] = grid.max_game.str[0].astype(int)
grid = grid.drop('max_game', axis = 1)

In [14]:
grid['score'] = grid['max']/grid['max_game_int']

In [15]:
from sklearn.preprocessing import minmax_scale
grid['scaled_score'] = minmax_scale(grid['score'], feature_range=(1, 5))

In [16]:
grid = grid.dropna()

In [17]:
# now we have a listing for each user pairing them with each game they play, what genre it belongs to, and how many people watched them play each game compared to the max viewers they ever got for a stream during the week we examined.

In [18]:
grid.groupby('user_name')['scaled_score'].count().reset_index().sort_values('scaled_score', ascending=False)[:5]

,user_name,scaled_score
27473,Fonbet_RocketLeague,378
77300,StreamerHouse,375
123375,luke4316live,333
110114,gaules,315
27472,Fonbet_RLH,270


In [19]:
grid.groupby('game_genres')['scaled_score'].count().reset_index().sort_values('scaled_score', ascending=False)[:5]

,game_genres,scaled_score
0,Action,211633
28,Shooter,165931
7,FPS,131286
23,RPG,118595
15,MMORPG,88901


In [20]:
grid.groupby('game_name')['scaled_score'].count().reset_index().sort_values('scaled_score', ascending=False)[:5]

,game_name,scaled_score
128,Grand Theft Auto V,37860
180,Minecraft,35715
243,Rocket League,35043
37,Black Desert Online,33723
97,Escape From Tarkov,33366


In [21]:
min_number_scores = 5
filter_users = grid['user_name'].value_counts() > min_number_scores
filter_users = filter_users[filter_users].index.tolist()

In [22]:
grid_new = grid[(grid['user_name'].isin(filter_users))]
print('The original data frame shape:\t{}'.format(grid.shape))
print('The new data frame shape:\t{}'.format(grid_new.shape))

The original data frame shape:	(1364027, 10)
The new data frame shape:	(1137673, 10)


In [23]:
# After reshaping the grid, we are going to extract the recommendations for game genre, game titles, and games similar to those already rated by the streamers as a three-pronged recommender approach.

In [24]:
import pickle

In [25]:
pickle.dump(grid_new, open("./Data/final_grid_06_07_19.pkl", "wb" ) )

In [26]:
grid_new = pickle.load( open( "./Data/final_grid_06_07_19.pkl", "rb" ) )

In [27]:
grid_new.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1137673 entries, 0 to 1387491
Data columns (total 10 columns):
user_name       1137673 non-null object
game_name       1137673 non-null object
game_genres     1137673 non-null object
language        1137673 non-null object
started_at      1137673 non-null object
viewer_count    1137673 non-null int64
max             1137673 non-null int64
max_game_int    1137673 non-null int64
score           1137673 non-null float64
scaled_score    1137673 non-null float64
dtypes: float64(2), int64(3), object(5)
memory usage: 95.5+ MB


In [28]:
grid_new.head(5)

,user_name,game_name,game_genres,language,started_at,viewer_count,max,max_game_int,score,scaled_score
0,龜狗,Grand Theft Auto V,Action,zh,2019-06-06 07:30:59,1735,1735,1735,1.00000,5.000000
1,龜狗,Grand Theft Auto V,Open World,zh,2019-06-06 07:30:59,1735,1735,1735,1.00000,5.000000
2,龜狗,Grand Theft Auto V,Driving/Racing Game,zh,2019-06-06 07:30:59,1735,1735,1735,1.00000,5.000000
3,龜狗,Grand Theft Auto V,Open World,zh,2019-06-06 07:30:59,1589,1589,1735,0.91585,4.663401
4,龜狗,Grand Theft Auto V,Action,zh,2019-06-06 07:30:59,1589,1589,1735,0.91585,4.663401


In [29]:
# preparing for genre recommendations based on viewership scores:

In [30]:
genres = grid_new.groupby(by = ['user_name', 'game_genres'])['scaled_score'].agg([np.mean])
games = grid_new.groupby(by = ['user_name', 'game_name'])['scaled_score'].agg([np.mean])

In [31]:
genres.head(5)

mean
user_name     game_genres          
00NothingLabs Fighting     2.421053
              Open World   1.578947
              RPG          1.578947
              Shooter      1.578947
00elu00       Action       4.000000

In [32]:
games.head(5)

mean
user_name     game_name                            
00NothingLabs Mortal Kombat 11             2.421053
              Tom Clancy's The Division 2  1.578947
00elu00       Dead by Daylight             4.666667
              Deathgarden                  2.000000
01joga        PUBG MOBILE                  3.750000

In [33]:
genres.columns = genres.columns.map(''.join)
games.columns = games.columns.map(''.join)

In [34]:
genres = genres.reset_index()
games = games.reset_index()

In [35]:
genres.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 181215 entries, 0 to 181214
Data columns (total 3 columns):
user_name      181215 non-null object
game_genres    181215 non-null object
mean           181215 non-null float64
dtypes: float64(1), object(2)
memory usage: 4.1+ MB


In [36]:
games.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 93606 entries, 0 to 93605
Data columns (total 3 columns):
user_name    93606 non-null object
game_name    93606 non-null object
mean         93606 non-null float64
dtypes: float64(1), object(2)
memory usage: 2.1+ MB


In [37]:
genres.head(5)

,user_name,game_genres,mean
0,00NothingLabs,Fighting,2.421053
1,00NothingLabs,Open World,1.578947
2,00NothingLabs,RPG,1.578947
3,00NothingLabs,Shooter,1.578947
4,00elu00,Action,4.000000


In [38]:
games.head(5)

,user_name,game_name,mean
0,00NothingLabs,Mortal Kombat 11,2.421053
1,00NothingLabs,Tom Clancy's The Division 2,1.578947
2,00elu00,Dead by Daylight,4.666667
3,00elu00,Deathgarden,2.000000
4,01joga,PUBG MOBILE,3.750000


In [39]:
#using Surprise to predict genres/games for a streamer based on their existing genre ratings:

In [40]:
import surprise
from surprise import Dataset, accuracy, Reader, NMF, NormalPredictor, BaselineOnly, CoClustering, SlopeOne, SVD, KNNBaseline
from surprise.model_selection import GridSearchCV, cross_validate, train_test_split

In [41]:
reader = Reader(rating_scale=(1, 5))
genre_data = Dataset.load_from_df(genres[['user_name', 'game_genres', 'mean']], reader)

In [42]:
game_data = Dataset.load_from_df(games[['user_name', 'game_name', 'mean']], reader)

In [43]:
benchmark = []
# Iterate over all algorithms
for algorithm in [SlopeOne(), NormalPredictor(), BaselineOnly(), NMF(), CoClustering(), SVD()]:


    # Perform cross validation
    results = cross_validate(algorithm, genre_data, measures=['RMSE'], cv=5, verbose=False)
    
    # Get results & append algorithm name
    tmp = pd.DataFrame.from_dict(results).mean(axis=0)
    tmp = tmp.append(pd.Series([str(algorithm).split(' ')[0].split('.')[-1]], index=['Algorithm']))
    benchmark.append(tmp)
    
pd.DataFrame(benchmark).set_index('Algorithm').sort_values('test_rmse')  

Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...


,test_rmse,fit_time,test_time
Algorithm,,,
SlopeOne,0.583921,0.531420,0.389072
NMF,0.657008,9.962431,0.221860
SVD,0.686441,6.620686,0.306897
BaselineOnly,0.736105,0.489835,0.192276
CoClustering,0.769125,6.987103,0.178664
NormalPredictor,1.103599,0.197514,0.246395


In [44]:
benchmark = []
# Iterate over all algorithms
for algorithm in [SlopeOne(), NormalPredictor(), BaselineOnly(), NMF(), CoClustering(), SVD()]:


    # Perform cross validation
    results = cross_validate(algorithm, game_data, measures=['RMSE'], cv=5, verbose=True)
    
    # Get results & append algorithm name
    tmp = pd.DataFrame.from_dict(results).mean(axis=0)
    tmp = tmp.append(pd.Series([str(algorithm).split(' ')[0].split('.')[-1]], index=['Algorithm']))
    benchmark.append(tmp)
    
pd.DataFrame(benchmark).set_index('Algorithm').sort_values('test_rmse')  

Evaluating RMSE of algorithm SlopeOne on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9628  0.9649  0.9627  0.9665  0.9715  0.9657  0.0033  
Fit time          0.42    0.44    0.44    0.43    0.41    0.43    0.01    
Test time         0.21    0.12    0.13    0.12    0.12    0.14    0.03    
Evaluating RMSE of algorithm NormalPredictor on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.1558  1.1509  1.1476  1.1568  1.1587  1.1540  0.0041  
Fit time          0.09    0.12    0.12    0.12    0.11    0.11    0.01    
Test time         0.21    0.10    0.10    0.09    0.08    0.12    0.05    
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Evaluating RMSE of algorithm BaselineOnly on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std   

,test_rmse,fit_time,test_time
Algorithm,,,
BaselineOnly,0.798890,0.301758,0.115608
SVD,0.803675,3.511643,0.132568
SlopeOne,0.965683,0.430204,0.138744
NMF,0.978632,6.402927,0.097301
CoClustering,1.031011,4.950078,0.150136
NormalPredictor,1.153988,0.108952,0.115581


In [45]:
# Based on the results, we will proceed with SlopeOne for genre predictions. More information about SlopeOne can be found here: https://arxiv.org/abs/cs/0702144

In [46]:
# Based on the results, we will proceed with BaselineOnly for game predictions.

In [47]:
algo_genre = SlopeOne()
cross_validate(algo_genre, genre_data, measures=['RMSE'], cv=7, verbose=True)

Evaluating RMSE of algorithm SlopeOne on 7 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Fold 6  Fold 7  Mean    Std     
RMSE (testset)    0.5703  0.5742  0.5734  0.5654  0.5614  0.5848  0.5719  0.5716  0.0068  
Fit time          0.50    0.60    0.55    0.54    0.55    0.56    0.54    0.55    0.03    
Test time         0.23    0.25    0.23    0.22    0.34    0.21    0.21    0.24    0.04    


{'test_rmse': array([0.57029229, 0.57422004, 0.57343479, 0.56542988, 0.5614175 ,
        0.58481514, 0.57188791]),
 'fit_time': (0.49828195571899414,
  0.5985500812530518,
  0.547184944152832,
  0.5449793338775635,
  0.5492870807647705,
  0.5629177093505859,
  0.5351979732513428),
 'test_time': (0.226593017578125,
  0.2510550022125244,
  0.22681212425231934,
  0.22183775901794434,
  0.33522605895996094,
  0.20784807205200195,
  0.20910286903381348)}

In [48]:
bsl_options = {'method': 'als',
               'n_epochs': 5,
               'reg_u': 12,
               'reg_i': 5
               }
algo_games = BaselineOnly(bsl_options=bsl_options)
cross_validate(algo_games, game_data, measures=['RMSE'], cv=7, verbose=True)

Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Evaluating RMSE of algorithm BaselineOnly on 7 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Fold 6  Fold 7  Mean    Std     
RMSE (testset)    0.8009  0.7926  0.7999  0.8008  0.7925  0.7992  0.7977  0.7977  0.0034  
Fit time          0.17    0.20    0.20    0.20    0.19    0.19    0.21    0.20    0.01    
Test time         0.07    0.07    0.07    0.07    0.07    0.07    0.07    0.07    0.00    


{'test_rmse': array([0.80085355, 0.79261895, 0.79994394, 0.80083377, 0.79254518,
        0.79919841, 0.7977099 ]),
 'fit_time': (0.1748201847076416,
  0.19878029823303223,
  0.20044803619384766,
  0.20410799980163574,
  0.19025087356567383,
  0.1910839080810547,
  0.20923995971679688),
 'test_time': (0.06641197204589844,
  0.07117915153503418,
  0.07161521911621094,
  0.07212090492248535,
  0.0654449462890625,
  0.06582403182983398,
  0.07269883155822754)}

In [49]:
genre_trainset, genre_testset = train_test_split(genre_data, test_size=0.25)
algo_genre = SlopeOne()
genre_predictions = algo_genre.fit(genre_trainset).test(genre_testset)
accuracy.rmse(genre_predictions)

RMSE: 0.5921


0.592118245254638

In [50]:
game_trainset, game_testset = train_test_split(game_data, test_size=0.25)
algo_games = BaselineOnly(bsl_options=bsl_options)
game_predictions = algo_games.fit(game_trainset).test(game_testset)
accuracy.rmse(game_predictions)

Estimating biases using als...
RMSE: 0.7953


0.7952873385219098

In [51]:
pickle.dump(algo_genre, open("./Data/SlopeOne_genre_model.pkl", "wb" ) )
algo_genre = pickle.load( open( "./Data/SlopeOne_genre_model.pkl", "rb" ) )

In [52]:
pickle.dump(algo_games, open("./Data/BaselineOnly_game_model.pkl", "wb" ) )
algo_games = pickle.load( open( "./Data/BaselineOnly_game_model.pkl", "rb" ) )

In [53]:
pickle.dump(genre_predictions, open("./Data/SlopeOne_genre_model_predictions.pkl", "wb" ) )
genre_predictions = pickle.load( open( "./Data/SlopeOne_genre_model_predictions.pkl", "rb" ) )

In [54]:
pickle.dump(game_predictions, open("./Data/BaselineOnly_game_model_predictions.pkl", "wb" ) )
game_predictions = pickle.load( open( "./Data/BaselineOnly_game_model_predictions.pkl", "rb" ) )

In [55]:
pickle.dump(genres, open("./Data/genres.pkl", "wb" ) )
genres = pickle.load( open( "./Data/genres.pkl", "rb" ) )

In [87]:
pickle.dump(games, open("./Data/games.pkl", "wb" ) )
games = pickle.load( open( "./Data/games.pkl", "rb" ) )

In [88]:
streamer_name = input('What is your streamer name? ')
streamer_genres = list(input ('Which game genres do you currently stream? ').split(', '))
streamer_games = list(input ('Which games do you currently steam? ').split(', '))

What is your streamer name? ninja
Which game genres do you currently stream? shooter
Which games do you currently steam? Fortnite


In [89]:
streamer_name, streamer_genres, streamer_games

('ninja', ['shooter'], ['Fortnite'])

In [90]:
#Making genre/game predictions for a specific streamer based on their user name:

In [91]:
genres.head()

,user_name,game_genres,mean
0,00NothingLabs,Fighting,2.421053
1,00NothingLabs,Open World,1.578947
2,00NothingLabs,RPG,1.578947
3,00NothingLabs,Shooter,1.578947
4,00elu00,Action,4.000000


In [92]:
def display_current_genres(streamer_name):
    user_genres = list(genres[genres['user_name']==streamer_name]['game_genres'])
    return user_genres
recorder_genres_list = display_current_genres(streamer_name)
full_genres = set(recorder_genres_list + streamer_genres)
full_genres

{'shooter'}

In [93]:
def display_current_games(streamer_name):
    user_games = list(games[games['user_name']==streamer_name]['game_name'])
    return user_games
recorder_games_list = display_current_games(streamer_name)
full_games = set(recorder_games_list + streamer_games)
full_games

{'Fortnite'}

####  Predicting Genres for Streamers (user-based similarities) ####

In [94]:
from collections import defaultdict

In [95]:
def get_top_n(predictions, n=10):
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n

In [96]:
iids_genre = genres['game_genres'].unique()
iids_genre_to_predict = np.setdiff1d(iids_genre, full_genres)

In [97]:
iids_game = games['game_name'].unique()
iids_game_to_predict = np.setdiff1d(iids_game, full_games)

In [98]:
iids_genre_to_predict

array(['Action', 'Adventure Game', 'Card & Board Game', 'Compilation',
       'Creative', 'Driving/Racing Game', 'Educational Game', 'FPS',
       'Fighting', 'Flight Simulator', 'Gambling Game', 'Hidden Objects',
       'Horror', 'IRL', 'Indie Game', 'MMORPG', 'MOBA', 'Metroidvania',
       'Open World', 'Other', 'Platformer', 'Point and Click', 'Puzzle',
       'RPG', 'RTS', 'Rhythm & Music Game', 'Roguelike', 'Series: Souls',
       'Shooter', 'Simulation', 'Sports Game', 'Stealth', 'Strategy',
       'Survival', 'Visual Novel'], dtype=object)

In [99]:
iids_game_to_predict

array(['7 Days to Die', 'A Hat in Time', 'A Plague Tale: Innocence',
       'A Way Out', 'ARK', 'ASMR', 'ATLAS', 'Agar.io',
       'Age of Empires II', 'Age of Mythology', 'Albion Online',
       'Alien: Isolation', 'Always On', 'American Truck Simulator',
       'Anno 1800', 'Apex Legends', 'Arena of Valor', 'Arma 3', 'Art',
       "Assassin's Creed II", "Assassin's Creed III",
       "Assassin's Creed Odyssey", 'Assetto Corsa Competizione',
       'Astroneer', 'Auto Chess', 'BaBa Is You', 'Bad North',
       'Banjo-Kazooie', 'Barotrauma', 'Battalion 1944', 'Battle Brothers',
       'Battlefield 1', 'Battlefield 4', 'Battlefield V', 'Beat Saber',
       'BioShock', 'Black Desert Online', 'Blackjack', 'Blood & Truth',
       'Bloodborne', 'Borderlands 2', 'Brawl Stars', 'Brawlhalla',
       'Call of Duty: Black Ops 4', 'Call of Duty: Black Ops II',
       'Call of Duty: Black Ops III', 'Call of Duty: Modern Warfare 3',
       'Call of Duty: Modern Warfare Remastered', 'Call of Duty: WW

In [100]:
genre_testset_personal = [[streamer_name, iid, 0.] for iid in iids_genre_to_predict]
game_testset_personal = [[streamer_name, iid, 0.] for iid in iids_game_to_predict]

In [101]:
genre_testset_personal

[['ninja', 'Action', 0.0],
 ['ninja', 'Adventure Game', 0.0],
 ['ninja', 'Card & Board Game', 0.0],
 ['ninja', 'Compilation', 0.0],
 ['ninja', 'Creative', 0.0],
 ['ninja', 'Driving/Racing Game', 0.0],
 ['ninja', 'Educational Game', 0.0],
 ['ninja', 'FPS', 0.0],
 ['ninja', 'Fighting', 0.0],
 ['ninja', 'Flight Simulator', 0.0],
 ['ninja', 'Gambling Game', 0.0],
 ['ninja', 'Hidden Objects', 0.0],
 ['ninja', 'Horror', 0.0],
 ['ninja', 'IRL', 0.0],
 ['ninja', 'Indie Game', 0.0],
 ['ninja', 'MMORPG', 0.0],
 ['ninja', 'MOBA', 0.0],
 ['ninja', 'Metroidvania', 0.0],
 ['ninja', 'Open World', 0.0],
 ['ninja', 'Other', 0.0],
 ['ninja', 'Platformer', 0.0],
 ['ninja', 'Point and Click', 0.0],
 ['ninja', 'Puzzle', 0.0],
 ['ninja', 'RPG', 0.0],
 ['ninja', 'RTS', 0.0],
 ['ninja', 'Rhythm & Music Game', 0.0],
 ['ninja', 'Roguelike', 0.0],
 ['ninja', 'Series: Souls', 0.0],
 ['ninja', 'Shooter', 0.0],
 ['ninja', 'Simulation', 0.0],
 ['ninja', 'Sports Game', 0.0],
 ['ninja', 'Stealth', 0.0],
 ['ninja', 'St

In [102]:
genre_predictions = algo_genre.test(genre_testset_personal)
game_predictions = algo_games.test(game_testset_personal)

In [103]:
genre_predictions[:3]

[Prediction(uid='ninja', iid='Action', r_ui=0.0, est=3.5672893506051544, details={'was_impossible': True, 'reason': 'User and/or item is unkown.'}),
 Prediction(uid='ninja', iid='Adventure Game', r_ui=0.0, est=3.5672893506051544, details={'was_impossible': True, 'reason': 'User and/or item is unkown.'}),
 Prediction(uid='ninja', iid='Card & Board Game', r_ui=0.0, est=3.5672893506051544, details={'was_impossible': True, 'reason': 'User and/or item is unkown.'})]

In [104]:
game_predictions[:3]

[Prediction(uid='ninja', iid='7 Days to Die', r_ui=0.0, est=3.5356776134169796, details={'was_impossible': False}),
 Prediction(uid='ninja', iid='A Hat in Time', r_ui=0.0, est=3.3132430447469505, details={'was_impossible': False}),
 Prediction(uid='ninja', iid='A Plague Tale: Innocence', r_ui=0.0, est=3.1197266151449288, details={'was_impossible': False})]

In [105]:
top_n_genres = get_top_n(genre_predictions)
top_n_games = get_top_n(game_predictions)

In [106]:
top_n_genres

defaultdict(list,
            {'ninja': [('Action', 3.5672893506051544),
              ('Adventure Game', 3.5672893506051544),
              ('Card & Board Game', 3.5672893506051544),
              ('Compilation', 3.5672893506051544),
              ('Creative', 3.5672893506051544),
              ('Driving/Racing Game', 3.5672893506051544),
              ('Educational Game', 3.5672893506051544),
              ('FPS', 3.5672893506051544),
              ('Fighting', 3.5672893506051544),
              ('Flight Simulator', 3.5672893506051544)]})

In [107]:
top_n_games

defaultdict(list,
            {'ninja': [('Silent Hill', 4.044002331526911),
              ('The Jackbox Party Pack 3', 4.011023346466942),
              ('Metin 2', 3.9545480102913326),
              ('Pokémon Ultra Sun/Ultra Moon', 3.936042687933252),
              ('Anno 1800', 3.933875242887823),
              ('Battlefield 1', 3.9240914744146758),
              ('Tabletop RPGs', 3.91431490039107),
              ('Dark Souls II', 3.909882161273136),
              ('Minecraft', 3.905512341203254),
              ('Danganronpa: Trigger Happy Havoc', 3.9052059116180633)]})

In [108]:
for uid, user_ratings in top_n_genres.items():
    print('For ' + uid + ', the recommended genres are:'+ str([iid for (iid, _) in user_ratings]))
genre_user_based_list = [iid for (iid, _) in user_ratings]
genre_user_based_list

For ninja, the recommended genres are:['Action', 'Adventure Game', 'Card & Board Game', 'Compilation', 'Creative', 'Driving/Racing Game', 'Educational Game', 'FPS', 'Fighting', 'Flight Simulator']


['Action',
 'Adventure Game',
 'Card & Board Game',
 'Compilation',
 'Creative',
 'Driving/Racing Game',
 'Educational Game',
 'FPS',
 'Fighting',
 'Flight Simulator']

In [109]:
for uid, user_ratings in top_n_games.items():
    print('For ' + uid + ', the recommended games are:'+ str([iid for (iid, _) in user_ratings]))

For ninja, the recommended games are:['Silent Hill', 'The Jackbox Party Pack 3', 'Metin 2', 'Pokémon Ultra Sun/Ultra Moon', 'Anno 1800', 'Battlefield 1', 'Tabletop RPGs', 'Dark Souls II', 'Minecraft', 'Danganronpa: Trigger Happy Havoc']


In [110]:
game_user_based_list = [iid for (iid, _) in user_ratings]
game_user_based_list

['Silent Hill',
 'The Jackbox Party Pack 3',
 'Metin 2',
 'Pokémon Ultra Sun/Ultra Moon',
 'Anno 1800',
 'Battlefield 1',
 'Tabletop RPGs',
 'Dark Souls II',
 'Minecraft',
 'Danganronpa: Trigger Happy Havoc']

#### Predicting Similar Genres Based on Current Genre/Game (item-based similarity) #### 

In [111]:
import io  # needed because of weird encoding of u.item file
from surprise import KNNBaseline
from surprise import Dataset
from surprise import get_dataset_dir

In [112]:
genre_group = grid_new.groupby(by = ['game_genres', 'user_name'])['scaled_score'].agg([np.mean])
game_group = grid_new.groupby(by = ['game_name', 'user_name'])['scaled_score'].agg([np.mean])

In [113]:
genre_group = genre_group.reset_index()
game_group = game_group.reset_index()

In [114]:
pickle.dump(genre_group, open("./Data/genre_group.pkl", "wb" ) )
genre_group = pickle.load( open( "./Data/genre_group.pkl", "rb" ) )

pickle.dump(game_group, open("./Data/game_group.pkl", "wb" ) )
game_group = pickle.load( open( "./Data/game_group.pkl", "rb" ) )

In [115]:
genre_group = pickle.load( open( "./Data/genre_group.pkl", "rb" ) )
reader = Reader(rating_scale=(1, 5))
genre_data = Dataset.load_from_df(genre_group[['user_name', 'game_genres', 'mean']], reader)
genre_trainset = genre_data.build_full_trainset()

game_group = pickle.load( open( "./Data/game_group.pkl", "rb" ) )
reader = Reader(rating_scale=(1, 5))
game_data = Dataset.load_from_df(game_group[['user_name', 'game_name', 'mean']], reader)
game_trainset = game_data.build_full_trainset()

In [116]:
#Fit the KNN algorithm to the data
sim_options = {'name': 'pearson_baseline',
               'shrinkage': 0, 'user_based': False  # no shrinkage
               }
algo_genre_group = KNNBaseline(sim_options=sim_options, verbose = False)
algo_genre_group.fit(genre_trainset)


algo_game_group = KNNBaseline(sim_options=sim_options, verbose = False)
algo_game_group.fit(game_trainset)

In [117]:
#produce the list of genres/games needed to be evaluated

genre_inner_id_list = []
for genre in full_genres:
    inner = algo_genre_group.trainset.to_inner_iid(genre)
    genre_inner_id_list.append(inner)

game_inner_id_list = []
for game in full_games:
    inner = algo_game_group.trainset.to_inner_iid(game)
    game_inner_id_list.append(inner)

ValueError: Item shooter is not part of the trainset.

In [118]:
# Retrieve inner ids of the nearest neighbors of the game in question.
genre_neighbors_list = []
for inner in genre_inner_id_list:
    genre_neighbors = algo_genre_group.get_neighbors(inner, k=3)
    genre_neighbors_list.append(genre_neighbors)

game_neighbors_list = []
for inner in game_inner_id_list:
    game_neighbors = algo_game_group.get_neighbors(inner, k=3)
    game_neighbors_list.append(game_neighbors)

NameError: name 'game_inner_id_list' is not defined

In [ ]:
print(genre_neighbors_list)
print(game_neighbors_list)

In [ ]:
# prioritize closest neighbors to all original genres/games mentioned

genre_final_list = []
for item in genre_neighbors_list:
    genre_final_list.append(item[0])
    genre_final_list.append(item[1])

game_final_list = []
for item in game_neighbors_list:
    game_final_list.append(item[0])
    game_final_list.append(item[1])

In [ ]:
print(genre_final_list)
print(game_final_list)

In [ ]:
#come up with a way to weigh the most frequent neighbors in all genres/games and combine with user-recommended ones.

In [ ]:
genres = [algo_genre_group.trainset.to_raw_iid(iiid) for iiid in set(genre_final_list)]

games = [algo_game_group.trainset.to_raw_iid(iiid) for iiid in set(game_final_list)]
print('The nearest neighbors of your current genres are:' + str(genres))
print('The nearest neighbors of your current games are:' + str(games))

In [ ]:
print (', '.join(item for item in set(genres + user_based_list)))

In [ ]:
# combined outputs of both methods to produce results common in both
print("We recommend you stream the following genres: " + ', '.join(item for item in set(genres + genre_user_based_list)))
print("We recommend you stream the following games: " + ', '.join(item for item in set(games + game_user_based_list)))
